# Predikcija ponašanja klijenata banke


## Sadržaj


1.  [Priprema okruženja](#priprema-okruzenja)
2.  [Priprema podataka](#priprema-podataka)
    1.  [Spljoštenje *golih* podataka](#spljostenje-golih-podataka)
    2.  [Dodavanje, izbacivanje i mijenjanje značajki](#dodavanje-izbacivanje-i-mijenjanje-znacajki),
3.  [Podjela](#podjela)
4.  [Konstrukcija modela](#konstrukcija-modela)
5.  [Inspekcija modela](#inspekcija-modela)
6.  [Predikcija](#predikcija)


## Uvod <a class="anchor" id="uvod"></a>

U suvremeno doba vrlo je popularna ideja *prognoziranja* u širokom spektru djelatnosti, a posebnu pažnju dobiva u onim područjima u kojima neočekivani ishod može imati katastrofalne posljedice. Da bi prognoziranje bilo teorijski moguće, nužna je ovisnost i korelacija između poznatih vrijednosti i onih koje želimo predvidjeti; a, da bi prognoziranje bilo praktično moguće, u pravilu za konstrukciju prognostičkog modela, pa čak i za njegovo prognoziranje, potrebno je osimsliti efikasni računalni algoritam koji automatizira matematički račun na kojemu se model osniva.

Budući da je mjerenje i otkrivanje ovisnosti (a posebno korelacije) u domeni matematike i osmišljavanje algoritma u domeni računarstva, kao studenti diplomskog studija *Računarstvo i matematika* na *Prirodoslovno-matematičkog fakulteta* *Sveučilišta u Zagrebu* kolege **Luka Naglić**, **Davor Penzar** i **Domagoj Ravlić** objedinili su se u tim *Petty* kao natjecatelji u natjecanju *Mozgalo* &mdash; studentskom natjecanju čija je tema *data science*, ove godine konkretno binarna klasifikacija na temelju nekih poznatih podataka. Istovremeno, ovo rješenje natjecateljskog zadatka služi im i kao rješenje projektnog zadatka za kolegij *Strojno učenje* na njihovom studiju.

### Problem

*Naručitelj* ovogodišnjeg zadatka za natjecanje *Mozgalo* je *Raiffeisenbank Hrvatska*, a kao zadatak postavili su predviđanje ponašanja svojih klijenata. Naime, problem je predvidjeti hoće li klijent prijevremeno raskinuti ugovor o kreditu odnosno depozitu &mdash; u slučaju kredita to bi značilo povrat cijelog duga prije isteka roka, a u slučaju depozita to bi značilo povlačenje svog uloženog novca prije dogovorenog datuma.


### Podaci

Sirovi podatci koji pripadaju takozvanom *trening-skupu* dani su tablicom u *.csv* formatu čiji svaki radak predstavlja izvještaj na kraju kvartala o točno jednom ugovoru. Svaki takav redak sadrži sljedeće informacije:

1.  *DATUM_IZVJESTAVANJA* &ndash; datum s kraja kvartala kada je promatrani izvještaj načinjen,
2.  *KLIJENT_ID* &ndash; jedinstvena numerička šifra klijenta,
3.  *OZNAKA_PARTIJE* &ndash; jedinstvena numerička šifra ugovora,
4.  *DATUM_OTVARANJA* &ndash; datum otvaranja ugovora,
5.  *PLANIRANI_DATUM_ZATVARANJA* &ndash; ugovoreni datum svršetka ugovora,
6.  *DATUM_ZATVARANJA* &ndash; stvarni datum svršetka ugovora,
7.  *UGOVORENI_IZNOS* &ndash; ugovoreni iznos u HRK,
8.  *STANJE_NA_KRAJU_PRETH_KVARTALA* &ndash; stanje ugovora (iznos u HRK) na kraju kvartala koji je prethodio kvartalu izvještaja,
9.  *STANJE_NA_KRAJU_KVARTALA* &ndash; stanje ugovora (iznos u HRK) na kraju kvartala izvještaja,
10. *VALUTA* &ndash; jedinstvena numerička šifra valute ugovora,
11. *VRSTA_KLIJENTA* &ndash; jedinstvena numerička šigra vrste klijenta,
12. *PROIZVOD* &ndash; jedinstvena šifra proizvoda,
13. *VRSTA_PROIZVODA* &ndash; jedinstvena šifra vrste proizvoda,
14. *VISINA_KAMATE* &ndash; kamatni koeficijent na kraju kvartala izvještaja,
15. *TIP_KAMATE* &ndash; jedinstvena šifra tipa kamate,
16. *STAROST* &ndash; starost klijenta na kraju kvartala izvještaja,
17. *PRIJEVREMENI_RASKID* &ndash; ciljna varijabla koja je istinita ako je ugovor raskinut prije ugovorenog datuma, a laž inače; **napomena:** dobivene vrijednosti treba korigirati [formulom](#eq:1).

Korekcija varijable *PRIJEVREMENI_RASKID*:
$$ \text{PRIJEVREMENI_RASKID} \left( x \right) \iff \text{DATUM_ZATVARANJA} \left( x \right) + 10 \, \text{dana} < \text{PLANIRANI_DATUM_ZATVARANJA} \left( x \right) \text{.} $$ <a class="anchor" id="eq:1"></a>


## Priprema okruženja <a class="anchor" id="priprema-okruzenja"></a>


In [1]:
# Stadardna Python biblioteka
import functools
import sys

# Ostali paketi
import catboost as cb
import hyperopt as ho
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.core.arrays.categorical import Categorical
from pandas.core.frame import DataFrame
from pandas.core.indexes.base import Index
from pandas.core.series import Series

# Postavi prikaz grafova unutar biljeznice.
%matplotlib inline

# Postavi stil grafova na `ggplot'.
plt.style.use('ggplot')

# Ispisi inacice koristenog softvera.
print('Python version: {0}'.format(sys.version))
print('Numpy version: {0}'.format(np.__version__))
print('Pandas version: {0}'.format(pd.__version__))
print('Matplotlib version: {0}'.format(mpl.__version__))
print('CatBoost version: {0}'.format(cb.__version__))
print('hyperopt version: {0}'.format(ho.__version__))


Python version: 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
Numpy version: 1.16.2
Pandas version: 0.24.2
Matplotlib version: 3.0.3
CatBoost version: 0.14.2
hyperopt version: 0.2


Portabilnosti radi, tablice ponekad spremamo u *.csv* formatu, ali, brzine radi, preferirat ćemo *.pkl* format. Stoga konstruirajmo funkciju koja *pametno* učitava tablicu &mdash; ako je *.pkl* datoteka dostupna, nju će učitati, a inače će iz *.csv* datoteke čitati tablicu sa zadanom interpretacijom datumskih i kategoričkih stupaca (kao i redaka zaglavlja i indeksa).


In [2]:
def ucitaj (ime, zaglavlje = 0, indeks = 0, datumi = None, kategorije = None):
    df = None

    try:
        df = pd.read_pickle('{0:s}.pkl'.format(ime))
    except FileNotFoundError:
        df = pd.read_csv(
            '{0:s}.csv'.format(ime),
            header = zaglavlje,
            index_col = indeks,
            parse_dates = datumi,
            infer_datetime_format = True if datumi is not None else None
        )
        for stupac in iter(kategorije if kategorije is not None else tuple()):
            vrijednosti = df[stupac].dropna().unique()
            if isinstance(vrijednosti, Categorical):
                vrijednosti = np.asarray(vrijednosti)
            try:
                if isinstance(vrijednosti, np.ndarray):
                    vrijednosti.sort()
                else:
                    vrijednosti = vrijednosti.sort_values()
            except (TypeError, ValueError, AttributeError):
                pass
            df[stupac] = df[stupac].astype(
                pd.api.types.CategoricalDtype(
                    categories = vrijednosti,
                    ordered = False
                )
            )
        df.to_pickle('{0:s}.pkl'.format(ime))

    return df


## Priprema podataka <a class="anchor" id="priprema-podataka"></a>


### Spljoštenje *golih* podataka <a class="anchor" id="spljostenje-golih-podataka"></a>


In [3]:
from script.pljoska import *


Za učitavanje treninškog *dataseta* koristi se poziv

```Python
df = ucitaj(
    '../dataNovi/validation_dataset_nan',
    zaglavlje = 0,
    indeks = None,
    datumi = [
        'DATUM_OTVARANJA',
        'PLANIRANI_DATUM_ZATVARANJA'
    ],
    kategorije = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'VALUTA',
        'VRSTA_KLIJENTA',
        'PROIZVOD',
        'VRSTA_PROIZVODA',
        'TIP_KAMATE'
    ]
)

spljosteni_df = None

try:
    spljosteni_df = ucitaj(
        '../dataNovi/validation_dataset_flat',
        zaglavlje = 0,
        indeks = 0,
        datumi = [
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        kategorije = [
            'KLIJENT_ID',
            'OZNAKA_PARTIJE',
            'PRVA_VALUTA',
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'PRVI_TIP_KAMATE',
            'ZADNJI_TIP_KAMATE'
        ]
    )
except FileNotFoundError:
    df.sort_values(
        by = [
            'OZNAKA_PARTIJE',
            'DATUM_OTVARANJA',
            'STAROST'
        ],
        ascending = True,
        inplace = True
    )
    pojednostavi_indeks(df)

    spljosteni_df = spljosti(df)
    kategoriziraj(spljosteni_df)

    df.sort_values(by = ['instance_id'], ascending = True, inplace = True)
    pojednostavi_indeks(df)

    spljosteni_df.to_csv('../dataNovi/validation_dataset_flat.csv')

```


In [4]:
df = ucitaj(
    '../dataNovi/validation_dataset_nan',
    zaglavlje = 0,
    indeks = None,
    datumi = [
        'DATUM_OTVARANJA',
        'PLANIRANI_DATUM_ZATVARANJA'
    ],
    kategorije = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'VALUTA',
        'VRSTA_KLIJENTA',
        'PROIZVOD',
        'VRSTA_PROIZVODA',
        'TIP_KAMATE'
    ]
)

spljosteni_df = None

try:
    spljosteni_df = ucitaj(
        '../dataNovi/validation_dataset_flat',
        zaglavlje = 0,
        indeks = 0,
        datumi = [
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        kategorije = [
            'KLIJENT_ID',
            'OZNAKA_PARTIJE',
            'PRVA_VALUTA',
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'PRVI_TIP_KAMATE',
            'ZADNJI_TIP_KAMATE'
        ]
    )
except FileNotFoundError:
    df.sort_values(
        by = [
            'OZNAKA_PARTIJE',
            'DATUM_OTVARANJA',
            'STAROST'
        ],
        ascending = True,
        inplace = True
    )
    pojednostavi_indeks(df)

    spljosteni_df = spljosti(df)
    kategoriziraj(spljosteni_df)

    df.sort_values(by = ['instance_id'], ascending = True, inplace = True)
    pojednostavi_indeks(df)

    spljosteni_df.to_csv('../dataNovi/validation_dataset_flat.csv')


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84879 entries, 0 to 84878
Data columns (total 14 columns):
instance_id                   84879 non-null int64
KLIJENT_ID                    84879 non-null category
OZNAKA_PARTIJE                84879 non-null category
DATUM_OTVARANJA               84879 non-null datetime64[ns]
PLANIRANI_DATUM_ZATVARANJA    84879 non-null datetime64[ns]
UGOVORENI_IZNOS               84879 non-null float64
VALUTA                        84879 non-null category
VRSTA_KLIJENTA                84879 non-null category
PROIZVOD                      84879 non-null category
VRSTA_PROIZVODA               84879 non-null category
VISINA_KAMATE                 83588 non-null float64
TIP_KAMATE                    84879 non-null category
STAROST                       84879 non-null int64
PRIJEVREMENI_RASKID           0 non-null float64
dtypes: category(7), datetime64[ns](2), float64(3), int64(2)
memory usage: 6.7 MB


In [6]:
spljosteni_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75704 entries, 0 to 75703
Data columns (total 22 columns):
instance_id                          75704 non-null int64
KLIJENT_ID                           75704 non-null category
OZNAKA_PARTIJE                       75704 non-null category
PRVI_DATUM_OTVARANJA                 75704 non-null datetime64[ns]
ZADNJI_DATUM_OTVARANJA               75704 non-null datetime64[ns]
PRVI_PLANIRANI_DATUM_ZATVARANJA      75704 non-null datetime64[ns]
ZADNJI_PLANIRANI_DATUM_ZATVARANJA    75704 non-null datetime64[ns]
VRSTA_KLIJENTA                       75704 non-null category
PRVA_STAROST                         75704 non-null int64
STAROST                              75704 non-null int64
ZADNJA_STAROST                       75704 non-null int64
VRSTA_PROIZVODA                      75704 non-null category
PROIZVOD                             75704 non-null category
PRVI_UGOVORENI_IZNOS                 75704 non-null float64
ZADNJI_UGOVORENI_IZNOS    

In [7]:
df.head()

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,0,800680,5743443,2010-11-26,2017-11-30,40000.00,1,1410,FL0801,A,9.70,C,33,NaN
1,1,588946,12374266,2016-11-15,2022-05-31,31000.00,1,1410,FL0801,A,7.25,B,35,NaN
2,2,410939,1434194,2007-12-10,2017-12-31,78000.00,1,1410,FL0801,A,8.45,B,43,NaN
3,3,418941,6410546,2011-07-19,2018-07-31,98823.63,2,1410,FL0801,A,7.57,B,51,NaN
4,4,1226823,6489686,2011-09-09,2021-09-30,100000.00,1,1410,FL0801,A,8.95,B,51,NaN


In [8]:
spljosteni_df.head()

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,PRVI_DATUM_OTVARANJA,ZADNJI_DATUM_OTVARANJA,PRVI_PLANIRANI_DATUM_ZATVARANJA,ZADNJI_PLANIRANI_DATUM_ZATVARANJA,VRSTA_KLIJENTA,PRVA_STAROST,STAROST,...,PROIZVOD,PRVI_UGOVORENI_IZNOS,ZADNJI_UGOVORENI_IZNOS,PRVA_VALUTA,ZADNJA_VALUTA,PRVI_TIP_KAMATE,ZADNJI_TIP_KAMATE,PRVA_VISINA_KAMATE,ZADNJA_VISINA_KAMATE,PRIJEVREMENI_RASKID
0,52621,1435613,12882219,2018-03-12,2018-03-12,2018-09-12,2018-09-12,1120,5,5,...,FL0900,2229301.20,2229301.20,2,2,A,A,2.60,2.60,0
1,34418,1338236,11125379,2015-12-09,2015-12-09,2016-06-10,2016-06-10,1410,77,77,...,TM0109,7602.09,7602.09,2,2,A,A,0.50,0.50,0
2,61106,1051081,1360749,2007-07-20,2007-07-20,2012-07-31,2012-07-31,1410,72,72,...,FL1100,24083.77,24083.77,1,1,C,C,9.45,9.45,0
3,42620,525064,6647031,2012-06-23,2012-06-23,2012-12-24,2012-12-24,1410,35,35,...,TM0109,50000.00,50000.00,1,1,A,A,3.70,3.70,0
4,20349,110067,6983538,2012-04-11,2012-04-11,2020-05-31,2020-05-31,1410,44,44,...,FL0801,74710.25,74710.25,2,2,B,B,8.66,8.66,0


### Dodavanje, izbacivanje i mijenjanje značajki <a class="anchor" id="dodavanje-izbacivanje-i-mijenjanje-znacajki"></a>


Pretpostavimo da nedostatak visine kamate možemo smatrati kao da je ona $ 0.00 \, \% $.


In [9]:
def ispuni_nedefinirane (df):
    df.PRVA_VISINA_KAMATE.fillna(0, inplace = True)
    df.ZADNJA_VISINA_KAMATE.fillna(0, inplace = True)

    return df


In [10]:
from script.ubaci_nove import *


In [11]:
def izbacivanje (df):
    stupci_za_izbaciti = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'PRVA_STAROST',
        'STAROST',
        'ZADNJA_STAROST',
        'PRVA_VALUTA',
        'PRVI_TIP_KAMATE',
        'PRVA_VISINA_KAMATE'
    ]

    df.drop(columns = stupci_za_izbaciti, errors = 'ignore', inplace = True)

    return df


Neke makroekonomske podatke i statistički izračunate vrijednosti pročitajmo iz spremljenih datoteka.


In [12]:
ekonomski_indikatori = ucitaj('../dataNovi/ekonomski_indikatori', 0, 0, None, None)

score_vk = ucitaj('../dataNovi/score_vrsta_klijenta', 0, 0, None, None)
score_p  = ucitaj('../dataNovi/score_proizvod', 0, 0, None, None)


In [13]:
def pripremi (df):
    return izbacivanje(
        dodavanje(
            ekonomija(ispuni_nedefinirane(df), ekonomski_indikatori),
            ekonomski_indikatori,
            score_vk,
            score_p
        )
    )


Nakon poziva

```Python
pripremi(spljosteni_df)

```

tablica `spljosteni_df` sadrzavat će stupce

    PRVI_DATUM_OTVARANJA                 datetime64[ns]
    ZADNJI_DATUM_OTVARANJA               datetime64[ns]
    PRVI_PLANIRANI_DATUM_ZATVARANJA      datetime64[ns]
    ZADNJI_PLANIRANI_DATUM_ZATVARANJA    datetime64[ns]
    VRSTA_KLIJENTA                       category
    VRSTA_PROIZVODA                      category
    PROIZVOD                             category
    PRVI_UGOVORENI_IZNOS                 float64
    ZADNJI_UGOVORENI_IZNOS               float64
    ZADNJA_VALUTA                        category
    ZADNJI_TIP_KAMATE                    category
    ZADNJA_VISINA_KAMATE                 float64
    TREND_bdp                            float64
    TREND_inflacija                      float64
    TREND_nezaposlenosti                 float64
    TREND_nafta                          float64
    STD_bdp                              float64
    STD_inflacija                        float64
    STD_nezaposlenosti                   float64
    STD_nafta                            float64
    PROSJEK_bdp                          float64
    PROSJEK_inflacija                    float64
    PROSJEK_nezaposlenosti               float64
    duljina_dani                         int64
    duljina_godine                       float64
    slozeni_kamatni                      float64
    slozeni_kamatni_po_danu              float64
    slozeni_kamatni_po_godini            float64
    jednostavni_kamatni                  float64
    jednostavni_kamatni_po_danu          float64
    jednostavni_kamatni_po_godini        float64
    visina_kamate_kvadrat                float64
    broj_godina_u_krizi                  int64
    krizne_puta_ukupno                   float64
    veci_krizni_dug                      float64
    optimisticne_godine                  int64
    mean_probit_kam                      float64
    mean_score_kam                       float64
    promjena_tipa_kamate                 int64
    promjena_visine_kamate               int64
    promjena_ugovorenog_iznosa           int64
    PRIJEVREMENI_RASKID                  int64

Ako je `spljosteni_df` dobiven spljoštenjem treninškog *dataseta*, sadržavat će još i stupac

    DATUM_ZATVARANJA                     datetime64[ns]

a, ako je dobiven spljoštenjem evaluacijskog *dataseta*, sadržavat će stupac

    instance_id                          int64

Stupci `PROSJEK_...` i `STD_...` predstavljaju proječnu vrijednost i standardnu devijaciju vrijednosti kroz vrijeme (planiranog, ugovorenog) trajanja ugovora respektivno, a `TREND_...` koeficijent smjera linearne regresije.

Složeni kamatni račun je, na žalost, pogrešno izračunat jer se za visinu kamate ne uzima ta vrijednost kao postotak (ne dijeli se sa $ 100 $), nego se ona uzima takva kakva jest &mdash; složeni kamatni račun je računat po formuli
$$ \text{složeni kamatni račun} = \left( 1 + \text{prva visina kamate} \right)^{\text{duljina (godine)}} \cdot \text{prvi ugovoreni iznos} \text{.} $$
Jednostavni kamatni račun je, doduše, točno izračunat formulom
$$ \text{jednostavni kamatni račun} = \left( 1 + \text{duljina (godine)} \cdot \frac{\text{zadnja visina kamate}}{100} \right) \cdot \text{zadnji ugovoreni iznos} \text{.} $$
Kamatni računi po danu odnosno po godini izračunati su jednostavno dijeljenjem dobivene vrijednosti kamatnog računa s trajanjem ugovora u odgovarajućoj mjernoj jedinici.

Veći krizni dug izračunat je po formuli
$$ 1.5 \cdot \text{broj godina u krizi} \cdot \text{jednostavni kamatni račun} \text{,} $$
a optimistične godine kao $ 1 $ ako je zadnji planirani datum zatvaranja unutar $ 2 $ godine od kraja krize (moguće je da će klijent tada otvaranjem novog kredita, s boljim uvjetima, zatvoriti postojeći kredit od kojeg nije preostalo mnogo za vratiti) i $ 0 $ inače.

Značajke `mean_score_kam` i `mean_probit_kam` posebno su zanimljive. Za svaku kategorijsku vrijednost stupca `PROIZVOD` i za svaku kategorijsku vrijednost stupca `VRSTA_KLIJENTA` prvo su (posebno) izračunati statistički izračunate *vjerojatnosti* da će ugovor s tom vrijednosti biti prijevremeno raskinut (točnije, izračunati su udjeli prijevremeno raskinutih ugovora), i to tako da su te vjerojatosti posebno izračunate unutar grupe vrste proizvoda *A* i grupe vrste proizvoda *L*. Zatim su, ponovo unutar svake od grupa vrste proizvoda, izračunati lokalni maksimumi udjela prijevremeno raskinutih ugovora po razredima visine kamate (razredi su dobiveni pozivom funkcije `matplotlib.pyplot.hist`), a tada se za svaku visinu kamate uzima njezina udaljenost od sredine razreda s maksimalnom vjerojatnosti prijevremenog raskida kojemu ta visina kamate *teži* (za vrstu proizvoda *A* su, na primjer, $ 3 $ lokalna maksimuma po visini kamate pa se ovisno o visini kamate računa udaljenost od nekog od ta $ 3 $ lokalna maksimuma). Konačna vrijednost značajke `mean_probit_kam` izračunata je po formuli
$$ \text{mean_probit_kam} = \log \left( k \left( \text{vrsta proizvoda} , \text{visina kamate} \right) \cdot \exp \left( \frac{ f \left( \text{vrsta proizvoda} , p_{\text{vrsta klijenta}} \right) + f \left( \text{vrsta proizvoda} , p_{\text{proizvod}} \right)}{2} \right) \right) \text{,} $$
gdje je $ k \left( \text{vrsta proizvoda} , \text{visina kamate} \right) $ recipročna vrijednost udaljenosti od odgovarajućeg lokalnog maksimuma visine kamate ako je vrsta proizvoda *A*, a inače udaljenost od odgovarajućeg lokalnog maksimuma visine kamate, i
$$ f \left( \text{vrsta proizvoda} , p \right) = \begin{cases} \Phi^{{- 1}} \left( p \right) & \text{vrsta proizvoda} = A \\ \Phi^{{- 1}} \left( 1 - p \right) & \text{inače} \end{cases} \text{,} $$
gdje je $ \Phi $ funkcija distribucije normalne slučajne varijable s očekivanjem $ 0 $ i varijancom $ 1 $. Ako je $ p $ za manje od $ \varepsilon $ udaljen od $ 0 $ ili $ 1 $, uzima se $ p = \varepsilon $ odnosno $ p = 1 - \varepsilon $ (u kodu se koristi $ \varepsilon = 10^{{- 6}} $). Za vrstu proizvoda *A* vrijednost značajke `mean_probit_kam` bit će *visoka* ako je po svim kriterijima (vrsta klijenta, proizvod, visina kamate) vrlo vjerojatno da će ugovor biti prijevremeno raskinut, a vrlo niska (po apsolutnoj vrijednosti velika, ali inače strogo manja od $ 0 $) ako je vrlo vjerojatno da ugovor ne će biti prijevremeno raskinut. Za vrstu proizvoda *L* raspored je obrnut. Značajka `mean_score_kam` računa se po sličnoj formuli, ali u definiciji funkcije $ f $ umjesto $ \Phi^{{- 1}} $ uzima se identiteta.


In [14]:
try:
    spljosteni_df = ucitaj(
        '../dataNovi/validation_dataset_ready',
        zaglavlje = 0,
        indeks = 0,
        datumi = [
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        kategorije = [
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'ZADNJI_TIP_KAMATE'
        ]
    )
except FileNotFoundError:
    pripremi(spljosteni_df)

    spljosteni_df.to_csv('../dataNovi/validation_dataset_ready.csv')


In [15]:
spljosteni_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75704 entries, 0 to 75703
Data columns (total 45 columns):
instance_id                          75704 non-null int64
PRVI_DATUM_OTVARANJA                 75704 non-null datetime64[ns]
ZADNJI_DATUM_OTVARANJA               75704 non-null datetime64[ns]
PRVI_PLANIRANI_DATUM_ZATVARANJA      75704 non-null datetime64[ns]
ZADNJI_PLANIRANI_DATUM_ZATVARANJA    75704 non-null datetime64[ns]
VRSTA_KLIJENTA                       75704 non-null category
VRSTA_PROIZVODA                      75704 non-null category
PROIZVOD                             75704 non-null category
PRVI_UGOVORENI_IZNOS                 75704 non-null float64
ZADNJI_UGOVORENI_IZNOS               75704 non-null float64
ZADNJA_VALUTA                        75704 non-null category
ZADNJI_TIP_KAMATE                    75704 non-null category
ZADNJA_VISINA_KAMATE                 75704 non-null float64
PRIJEVREMENI_RASKID                  75704 non-null int64
TREND_bdp             

Nakon što vrijednosti datuma više nisu potrebne kao vrijednosti datuma, numerificiramo ih pozivom funkcije `pandas.to_numeric` i dijeljenjem s $ 10^{12} $.


In [16]:
def numerificiraj_datume (df):
    df.PRVI_DATUM_OTVARANJA = pd.to_numeric(
        df.PRVI_DATUM_OTVARANJA
    ) / 1.0e+12
    df.ZADNJI_DATUM_OTVARANJA = pd.to_numeric(
        df.ZADNJI_DATUM_OTVARANJA
    ) / 1.0e+12
    df.PRVI_PLANIRANI_DATUM_ZATVARANJA = pd.to_numeric(
        df.PRVI_PLANIRANI_DATUM_ZATVARANJA
    ) / 1.0e+12
    df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA = pd.to_numeric(
        df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA
    ) / 1.0e+12

    return df


## Podjela <a class="anchor" id="podjela"></a>


Za predikciju su konstruirana $ 3 $ modela:

1.  model za predikciju ugovora vrste proizvoda *A* zadnjeg planiranog datuma zatvaranja do $ 6 $. listopada $ 2016 $. godine,
2.  model za predikciju ugovora vrste proizvoda *L* zadnjeg planiranog datuma zatvaranja do $ 6 $. listopada $ 2016 $. godine,
3.  model za predikciju ugovora zadnjeg planiranog datuma zatvaranja nakon $ 6 $. listopada $ 2016 $. godine.

Ove posljednje zovemo *zeznutima* iz sljedećih razloga:

1.  vjerojatno su ekonomske prilike do $ 2016 $. godine, kao godine nakon zadnje velike krize, danas dovoljno jasne i iskristalizirane da su njihovi utjecaji dobro predvidljivi (osim određenih predikcija od $ 2019 $. godine nadalje, za neke makroekonoske pokazatelje ni nemamo dovoljno podataka za računanje prosječnih vrijednsoti, standardne devijacije i trendova),
2.  planirani datum zatvaranja nakon zadnjeg datuma izvještavanja ($ 31 $. prosinca $ 2018 $. godine) mogu stvarati probleme: čak i ako bi po nekim pokazateljima ugovor vjerojatno bio prijevremeno raskinut, taj prijevremeni raskid možda se još nije ni dogodio jer je ugovor ugovoren do, na primjer, $ 2040 $. godine &mdash; u tom slučaju model mora predviđati vrijednost u bazi podataka banke, a ne stvarnu vjerojatnost hoće li ugovor biti prijevremeno raskinut (što bi se možda desilo tek $ 2030 $. godine, ali ne još).


In [17]:
def podijeli (df):
    granica = pd.Timestamp('2019-10-06')

    df_zez = df.loc[df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA > granica].copy()
    df = df.drop(index = df_zez.index).copy()

    df.reset_index(drop = True, inplace = True)
    df_zez.reset_index(drop = True, inplace = True)

    po_vrsti_proizvoda = df.groupby('VRSTA_PROIZVODA')
    po_vrsti_proizvoda_zez = df_zez.groupby('VRSTA_PROIZVODA')

    df_A = po_vrsti_proizvoda.get_group('A').copy()
    df_L = po_vrsti_proizvoda.get_group('L').copy()
    df_A_zez = po_vrsti_proizvoda_zez.get_group('A').copy()
    df_L_zez = po_vrsti_proizvoda_zez.get_group('L').copy()

    del df
    del df_zez

    df_A.reset_index(drop = True, inplace = True)
    df_L.reset_index(drop = True, inplace = True)
    df_A_zez.reset_index(drop = True, inplace = True)
    df_L_zez.reset_index(drop = True, inplace = True)

    df_A.drop(columns = ['VRSTA_PROIZVODA'])
    df_L.drop(columns = ['VRSTA_PROIZVODA'])

    return (df_A, df_L, df_A_zez, df_L_zez)


Tablicu dijelimo pozivom

```Python
df_A, df_L, df_A_zez, df_L_zez = podijeli(spljosteni_df)

```

Modeli nadalje *očekuju* numerificirane datume (funkcijom `numerificiraj_datume`), pa su nužni i pozivi

```Python
numerificiraj_datume(df_A)
numerificiraj_datume(df_L)
numerificiraj_datume(df_A_zez)
numerificiraj_datume(df_L_zez)

```


In [18]:
df_A, df_L, df_A_zez, df_L_zez = podijeli(spljosteni_df)

df_A = numerificiraj_datume(df_A)
df_L = numerificiraj_datume(df_L)
df_A_zez = numerificiraj_datume(df_A_zez)
df_L_zez = numerificiraj_datume(df_L_zez)


## Konstrukcija modela <a class="anchor" id="konstrukcija-modela"></a>


Model konstruiramo kao objekt klase `catboost.CatBoostClassifier`, a njegove *hiperparametre* (argumente konstruktora) optimiziramo paketom *hyperopt*.

Funkcija za takvu optimizaciju može, na primjer, biti sljedeća:

```Python
def optimizacija (
    space,
    train_X,
    train_y,
    test_X,
    test_y,
    cat_stupci,
    loss
):
    if space['learning_rate'] < 0:
        return {'status' : ho.STATUS_FAIL}
    if space['border_count'] < 1 or space['border_count'] > 254:
        return {'status' : ho.STATUS_FAIL}

    train_pool = cb.Pool(train_X, train_y, cat_features = cat_stupci)
    test_pool = cb.Pool(test_X, test_y, cat_features = cat_stupci)

    model = cb.CatBoostClassifier(
        iterations = 600,
        learning_rate = space['learning_rate'],
        depth = 9,
        l2_leaf_reg = space['l2_leaf_reg'],
        border_count = space['border_count'],
        od_type = space['od_type'],
        leaf_estimation_method = space['leaf_estimation_method'],
        random_seed = np.maximum(np.abs(space['random_seed']), 1),
        random_strength = space['random_strength'],
        bagging_temperature = space['bagging_temperature'],
        task_type ='GPU',
        sampling_unit = space['sampling_unit'],
    )

    model.fit(train_pool, eval_set = test_pool, verbose = False, plot = False)

    return {'loss' : 1 - loss(model, test_X, test_y), 'status' : ho.STATUS_OK}

```

Varijabla `cat_stupci` lista je imena stupaca čije vrijednosti su kategoričkog tipa, a `train_X`, `train_y`, `test_X` i `test_y` vjerojatno su dovoljno jasne same po sebi &mdash; dobivene su podjelom originalnog treninškof *dataseta* na podskup koji služi za trening i podskup koji služi za evaluaciju treninga (naravno, stupac `DATUM_ZATVARANJA` je izbačen).

Pretpostavimo da model želimo optimizirati tako da maksimiziramo njegovu točnost. U tu svrhu definiramo funkciju `loss` koja vraća $ 1 - \text{točnost} \left( \text{model} \right) $.

```Python
def loss (model, X, y):
    return 1 - (model.predict(X) == y).astype(float).sum() / X.shape[0]

```

Osim ovoga, mogli smo konstruirati i funkcije koje vraćaju $ 1 - \text{preciznost} \left( \text{model} \right) $, $ 1 - \text{odziv} \left( \text{model} \right) $, $ 1 - F_{1} \left( \text{model} \right) $ ili nešto sasvim četvrto/peto. Bitno je da je vrijednost bliska $ 0 $ ako je model zadovoljavajuće dobar, a bliska $ 1 $ ako je loš.

Optimizacija hiperparametara u predstavljenom okruženju može se provesti sljedećim pozivima (najbolje vrijednosti pronađene u `max_evals` iteracija &mdash; v. na dnu `best = ho.fmin(...)` &mdash; *spremljene* su u objektu `best`):

```Python
space = {
    'learning_rate' : ho.hp.uniform('learning_rate', 0.09, 0.95),
    'l2_leaf_reg' : ho.hp.qlognormal('l2_leaf_reg', 1, 5, 0.5),
    'border_count': ho.hp.randint('border_count', 300),
    'od_type' : ho.hp.choice('od_type', ['IncToDec', 'Iter']),
    'leaf_estimation_method' : ho.hp.choice(
        'leaf_estimation_method',
        ['Newton', 'Gradient']
    ),
    'random_seed' : ho.hp.randint('random_seed', 1000),
    'random_strength' : ho.hp.lognormal('random_strength', 1, 1),
    'bagging_temperature' : ho.hp.lognormal('bagging_temperature', 1, 1),
    'sampling_unit' : ho.hp.choice('sampling_unit', ['Group', 'Object'])
}

trials = ho.Trials()

best = ho.fmin(
    fn = functools.partial(
        optimizacija,
        train_X = train_X,
        train_y = train_y,
        test_X = test_X,
        test_y = test_y,
        cat_stupci = cat_stupci,
        loss = loss
    ),
    space = space,
    algo = ho.tpe.suggest,
    max_evals = 12,
    trials = trials
)

```

Nakon provedene optimizacije, model se sada trenira pozivima

```Python
train_pool = cb.Pool(train_X, train_y, cat_features = cat_stupci)
test_pool = cb.Pool(test_X, test_y, cat_features = cat_stupci)

model = cb.CatBoostClassifier(
    iterations = 1000,
    learning_rate = best['learning_rate'],
    depth = 9,
    l2_leaf_reg = best['l2_leaf_reg'],
    border_count = best['border_count'],
    od_type = best['od_type'],
    leaf_estimation_method = best['leaf_estimation_method'],
    random_seed = np.maximum(np.abs(best['random_seed']), 1),
    random_strength = best['random_strength'],
    bagging_temperature = best['bagging_temperature'],
    task_type ='GPU',
    sampling_unit = best['sampling_unit'],
)

model.fit(
    train_pool,
    eval_set = test_pool,
    verbose = False,
    plot = True,
    early_stopping_rounds = 50
)

```

U stvarnom treniranju modela (ne u optimizaciji hiperparametara) broj iteracija povećali smo na $ 1000 $, ali smo postavili `early_stopping_rounds` na $ 50 $ &mdash; ako u $ 50 $ iteracija vrijednost *loss*-funkcije na testnom *datasetu* nije poboljšana, treniranje se prekida. Model koristi postavu iz one iteracije u kojoj je vrijednost *loss*-funkcije na testnom *datasetu* bila najniža, čak i ako `early_stopping_rounds` ne postavimo i dopustimo da se model *pretrenira* velikim brojem iteracija.

Konačno model možemo spremiti pozivom

```Python
model.save('istrenirani_model')

```

Argument metode `catboost.CatBoostClassifier.save` je lokacija datoteke u koju model želimo *spremiti*. Analogno tada model učitavamo iz datoteke pozivom

```Python
model.load('istrenirani_model')

```


## Inspekcija modela <a class="anchor" id="inspekcija-modela"></a>


In [19]:
def tezine (model):
    return pd.DataFrame(
        data = model.get_feature_importance(),
        index = model.feature_names_,
        columns = ['Tezine']
    ).sort_values(by = 'Tezine', ascending = False)


Učitajmo postojeće modele.


In [20]:
model_A = cb.CatBoostClassifier()
model_L = cb.CatBoostClassifier()
model_zez = cb.CatBoostClassifier()


In [21]:
model_A.load_model('../modeli/model_kredit')
model_L.load_model('../modeli/model_stednja')
model_zez.load_model('../modeli/model_zez')


Provjerimo značajnosti značajki na temelju kojih modeli *odlučuju*. Svakoj značajki pridružena je vrijednost od $ 0 $ do $ 100 $, a zbroj tih vrijednosti iznosi $ 100 $ &mdash; ta vrijednost pokazuje udio (u postotku) značajnosti značajke. Time je rješenje, očitio, interpretabilno.


In [22]:
tezine(model_A)


,Tezine
ZADNJI_PLANIRANI_DATUM_ZATVARANJA,40.604931
PRVI_PLANIRANI_DATUM_ZATVARANJA,27.202793
PROIZVOD,5.036251
ZADNJI_DATUM_OTVARANJA,4.399511
jednostavni_kamatni_po_danu,4.395798
broj_godina_u_krizi,3.575990
ZADNJI_TIP_KAMATE,3.442417
duljina_dani,3.246124
mean_score_kam,2.498227
mean_probit_kam,1.442437


In [23]:
tezine(model_L)


,Tezine
ZADNJI_PLANIRANI_DATUM_ZATVARANJA,17.618267
PRVI_PLANIRANI_DATUM_ZATVARANJA,13.050462
duljina_dani,8.848077
ZADNJI_DATUM_OTVARANJA,7.809331
ZADNJA_VALUTA,6.308917
mean_probit_kam,6.110480
krizne_puta_ukupno,6.043843
PRVI_DATUM_OTVARANJA,5.923200
ZADNJA_VISINA_KAMATE,5.068478
STD_inflacija,5.043186


In [24]:
tezine(model_zez)


,Tezine
mean_score_kam,20.898619
ZADNJI_PLANIRANI_DATUM_ZATVARANJA,14.588239
PRVI_PLANIRANI_DATUM_ZATVARANJA,10.122368
PROIZVOD,6.782310
ZADNJI_TIP_KAMATE,6.722419
jednostavni_kamatni_po_danu,5.750835
PRVI_DATUM_OTVARANJA,5.739603
ZADNJA_VALUTA,4.523361
STD_nezaposlenosti,4.086336
veci_krizni_dug,3.798000


## Predikcija <a class="anchor" id="predikcija"></a>


*Zeznute* ugovore možda nije dovoljno predviđati modelom za *zeznute* ugovore. Srećom, klasifikatori klase `catboost.CatBoostClassifier` mogu, osim konačne klasifikacije, vratiti vjerojatnost pripadnosti. Kombiniranjem vjerojatnosti dvaju (ili čak više, ali ovdje to nije potrebno jer nemamo *više* modela) modela konačnu klasifikaciju možemo sami izračunati.


In [25]:
def kombinacija (model_1, model_2, X, combo):
    return np.argmax(
        combo(
            model_1.predict_proba(X[model_1.feature_names_]),
            model_2.predict_proba(X[model_2.feature_names_])
        ),
        axis = 1
    )


Predikciju za *ne-zeznute* ugovore provodimo pozivom

```Python
df_A['PRIJEVREMENI_RASKID'] = model_A.predict(df_A[model_A.feature_names_])
df_L['PRIJEVREMENI_RASKID'] = model_L.predict(df_L[model_L.feature_names_])

```

Za predikciju *zeznutih* ugovora prvo definiramo neku kombinirajuću funkciju

```Python
combo_1 = lambda a, b : a
combo_2 = lambda a, b : b
combo_1_plus_2 = lambda a, b : a + b
combo_1_mul_2 = lambda a, b : a * b

```

Treća i četvrta funkcija su zapravo simplifikacija aritmetičke i geometrijske sredine bez težina (dijeljenje s $ 2 $ ili korijenovanje ne će utjecati na maksimume i minimume). Osim tih sredina, mogli smo definirati i neku treću sredinu, a mogli smo definirati i neku težinsku sredinu.

Pretpostavimo da smo varijable `combo_A_zez` i `combo_L_zez` fiksirali na neke od spomenutih funkcija (eksplicitno napisanih ili teoretski spomenutih). Sada predikciju zeznutih ugovora provodimo pozivima

```Python
df_A_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_A,
    model_zez,
    df_A_zez,
    combo_A_zez
)
df_L_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_L,
    model_zez,
    df_L_zez,
    combo_L_zez
)

```

Daljnji kod pojednostavimo reindeksiranjem podskupova *dataseta*

```Python
df_A.index = df_A.instance_id
df_L.index = df_L.instance_id
df_A_zez.index = df_A_zez.instance_id
df_L_zez.index = df_L_zez.instance_id

```

Uvrštavanje predikcije u originalni *dataset* dalje provodimo pozivima

```Python
df['PRIJEVREMENI_RASKID'] = 0

df.loc[df_A.index, 'PRIJEVREMENI_RASKID'] = df_A.PRIJEVREMENI_RASKID
df.loc[df_L.index, 'PRIJEVREMENI_RASKID'] = df_L.PRIJEVREMENI_RASKID
df.loc[df_A_zez.index, 'PRIJEVREMENI_RASKID'] = df_A_zez.PRIJEVREMENI_RASKID
df.loc[df_L_zez.index, 'PRIJEVREMENI_RASKID'] = df_L_zez.PRIJEVREMENI_RASKID

df.PRIJEVREMENI_RASKID = df.PRIJEVREMENI_RASKID.map({0 : 'N', 1 : 'Y'})

```


In [26]:
df['PRIJEVREMENI_RASKID'] = 0


In [27]:
df_A['PRIJEVREMENI_RASKID'] = model_A.predict(df_A[model_A.feature_names_])
df_L['PRIJEVREMENI_RASKID'] = model_L.predict(df_L[model_L.feature_names_])


In [28]:
combo_A_zez = lambda a, b : b
combo_L_zez = lambda a, b : 0.01 * a ** 1.5 + b ** 1.5


In [29]:
df_A_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_A,
    model_zez,
    df_A_zez,
    combo_A_zez
)
df_L_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_L,
    model_zez,
    df_L_zez,
    combo_L_zez
)


In [30]:
df_A.index = df_A.instance_id
df_L.index = df_L.instance_id
df_A_zez.index = df_A_zez.instance_id
df_L_zez.index = df_L_zez.instance_id


In [31]:
df.loc[df_A.index, 'PRIJEVREMENI_RASKID'] = df_A.PRIJEVREMENI_RASKID
df.loc[df_L.index, 'PRIJEVREMENI_RASKID'] = df_L.PRIJEVREMENI_RASKID
df.loc[df_A_zez.index, 'PRIJEVREMENI_RASKID'] = df_A_zez.PRIJEVREMENI_RASKID
df.loc[df_L_zez.index, 'PRIJEVREMENI_RASKID'] = df_L_zez.PRIJEVREMENI_RASKID


In [32]:
df.PRIJEVREMENI_RASKID.astype(float).sum() / df.shape[0]


0.5520918012700432

In [33]:
df.PRIJEVREMENI_RASKID = df.PRIJEVREMENI_RASKID.map({0 : 'N', 1 : 'Y'})


In [34]:
df.to_csv('student.csv')
